In [ ]:
%config IPCompleter.greedy=True

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import variational_autoencoder as vae

print("Tensor Flow version {}".format(tf.__version__))



D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensor Flow version 1.8.0


In [2]:
#test utils
import weapon_data as weapons

def print_decoded_tensors_as_dict(weapon_data, array_of_tensors):
    genDict = {}
    
    for tensor in array_of_tensors:
        decoded, _ = weapon_data.decode_processed_tensor(tensor)
        
        for key, value in decoded.items():
            if key not in genDict:
                genDict[key] = []
            genDict[key].append(value)
    
    for key, value in genDict.items():
            print(key, "=", value)
            
            
def get_weapon_data():
    return weapons.get_data(2, 14, 0)

In [3]:
network_architecture = \
    dict(n_input=0, #set it in with scope
         n_hidden_1=26,
         n_hidden_2=12,
         n_z=2)  

learning_rate = 0.01
optimizer =  tf.train.RMSPropOptimizer(learning_rate)
transfer_fct = tf.nn.elu
num_epochs = 10000
batch_size = 4
epoch_debug_step = 1
    
with tf.Session() as sess:
    train_data, test_data = get_weapon_data()
    network_architecture['n_input'] = train_data.num_features
  
    network = vae.get_new_trained(sess, train_data, network_architecture, optimizer, 
                              transfer_fct, batch_size, num_epochs, epoch_debug_step)


Epoch:0001 - Cost:22.148115233
Epoch:0002 - Cost:17.096423893
Epoch:0003 - Cost:16.376204934
Epoch:0004 - Cost:16.166729965
Epoch:0005 - Cost:16.008140173
Epoch:0006 - Cost:15.011937059
Epoch:0007 - Cost:14.843593582
Epoch:0008 - Cost:13.209105694
Epoch:0009 - Cost:14.060888816
Epoch:0010 - Cost:13.410815697
Epoch:0011 - Cost:13.478237723
Epoch:0012 - Cost:11.894908965
Epoch:0013 - Cost:13.765849226
Epoch:0014 - Cost:12.154480296
Epoch:0015 - Cost:11.415428072
Epoch:0016 - Cost:11.489777152
Epoch:0017 - Cost:10.411790427
Epoch:0018 - Cost:12.220169878
Epoch:0019 - Cost:10.321895209
Epoch:0020 - Cost:11.374430469
Epoch:0021 - Cost:11.931075659
Epoch:0022 - Cost:9.797048764
Epoch:0023 - Cost:10.484979104
Epoch:0024 - Cost:10.788577553
Epoch:0025 - Cost:10.275939701
Epoch:0026 - Cost:10.452860149
Epoch:0027 - Cost:11.478922161
Epoch:0028 - Cost:10.051008179
Epoch:0029 - Cost:10.833278341
Epoch:0030 - Cost:9.810425916
Epoch:0031 - Cost:10.690809340
Epoch:0032 - Cost:9.537024505
Epoch:0033 

Epoch:0273 - Cost:8.126120830
Epoch:0274 - Cost:8.220213072
Epoch:0275 - Cost:7.877391860
Epoch:0276 - Cost:8.108532162
Epoch:0277 - Cost:8.310687703
Epoch:0278 - Cost:8.116706533
Epoch:0279 - Cost:8.249504825
Epoch:0280 - Cost:8.385620237
Epoch:0281 - Cost:7.972327405
Epoch:0282 - Cost:7.641165245
Epoch:0283 - Cost:8.385776009
Epoch:0284 - Cost:7.993785753
Epoch:0285 - Cost:8.174883850
Epoch:0286 - Cost:8.620729266
Epoch:0287 - Cost:7.857251640
Epoch:0288 - Cost:8.163654958
Epoch:0289 - Cost:7.658093580
Epoch:0290 - Cost:8.682561965
Epoch:0291 - Cost:7.764807724
Epoch:0292 - Cost:8.076920247
Epoch:0293 - Cost:8.167099645
Epoch:0294 - Cost:7.932663279
Epoch:0295 - Cost:7.798065561
Epoch:0296 - Cost:8.297341099
Epoch:0297 - Cost:8.230363951
Epoch:0298 - Cost:8.151485090
Epoch:0299 - Cost:8.196198381
Epoch:0300 - Cost:7.928902018
Epoch:0301 - Cost:8.357846463
Epoch:0302 - Cost:8.144018293
Epoch:0303 - Cost:8.071207392
Epoch:0304 - Cost:8.281067841
Epoch:0305 - Cost:8.582390042
Epoch:0306

Epoch:0550 - Cost:7.552198508
Epoch:0551 - Cost:8.070451256
Epoch:0552 - Cost:7.833276118
Epoch:0553 - Cost:8.240866969
Epoch:0554 - Cost:7.978858768
Epoch:0555 - Cost:8.078183827
Epoch:0556 - Cost:7.691896844
Epoch:0557 - Cost:7.869250831
Epoch:0558 - Cost:8.397879368
Epoch:0559 - Cost:8.073219615
Epoch:0560 - Cost:7.814514851
Epoch:0561 - Cost:7.939775707
Epoch:0562 - Cost:8.176061585
Epoch:0563 - Cost:7.582595390
Epoch:0564 - Cost:8.901083473
Epoch:0565 - Cost:7.542472224
Epoch:0566 - Cost:7.825857448
Epoch:0567 - Cost:7.907908147
Epoch:0568 - Cost:8.405753714
Epoch:0569 - Cost:7.416069211
Epoch:0570 - Cost:8.168597019
Epoch:0571 - Cost:8.104990816
Epoch:0572 - Cost:7.936337448
Epoch:0573 - Cost:8.074227611
Epoch:0574 - Cost:7.788454446
Epoch:0575 - Cost:8.290903317
Epoch:0576 - Cost:7.255624921
Epoch:0577 - Cost:7.947164115
Epoch:0578 - Cost:8.377310625
Epoch:0579 - Cost:7.924243461
Epoch:0580 - Cost:8.255568151
Epoch:0581 - Cost:8.026623478
Epoch:0582 - Cost:8.832676309
Epoch:0583

Epoch:0825 - Cost:7.837408832
Epoch:0826 - Cost:8.076654194
Epoch:0827 - Cost:8.532676637
Epoch:0828 - Cost:7.597836157
Epoch:0829 - Cost:7.702115862
Epoch:0830 - Cost:7.799564437
Epoch:0831 - Cost:7.819366365
Epoch:0832 - Cost:7.821264597
Epoch:0833 - Cost:7.679832496
Epoch:0834 - Cost:8.188082800
Epoch:0835 - Cost:7.451948827
Epoch:0836 - Cost:7.688957965
Epoch:0837 - Cost:8.372626342
Epoch:0838 - Cost:7.942817778
Epoch:0839 - Cost:7.457174031
Epoch:0840 - Cost:9.015280851
Epoch:0841 - Cost:7.708480279
Epoch:0842 - Cost:8.222075567
Epoch:0843 - Cost:7.476955159
Epoch:0844 - Cost:8.149294500
Epoch:0845 - Cost:8.213174054
Epoch:0846 - Cost:7.771917568
Epoch:0847 - Cost:7.600458430
Epoch:0848 - Cost:7.731320118
Epoch:0849 - Cost:8.488625324
Epoch:0850 - Cost:7.832588782
Epoch:0851 - Cost:7.847402197
Epoch:0852 - Cost:7.891051255
Epoch:0853 - Cost:7.567045182
Epoch:0854 - Cost:8.317047495
Epoch:0855 - Cost:8.514530662
Epoch:0856 - Cost:8.403244439
Epoch:0857 - Cost:8.094373282
Epoch:0858

Epoch:1100 - Cost:8.267105057
Epoch:1101 - Cost:8.751453325
Epoch:1102 - Cost:8.344248719
Epoch:1103 - Cost:7.525941383
Epoch:1104 - Cost:9.481376333
Epoch:1105 - Cost:8.407755469
Epoch:1106 - Cost:8.187493978
Epoch:1107 - Cost:8.825309829
Epoch:1108 - Cost:7.889690880
Epoch:1109 - Cost:9.166458475
Epoch:1110 - Cost:9.325574755
Epoch:1111 - Cost:8.321747157
Epoch:1112 - Cost:7.840394673
Epoch:1113 - Cost:8.604212123
Epoch:1114 - Cost:7.976107049
Epoch:1115 - Cost:9.150955531
Epoch:1116 - Cost:8.670741765
Epoch:1117 - Cost:9.334588314
Epoch:1118 - Cost:8.705620187
Epoch:1119 - Cost:8.657054000
Epoch:1120 - Cost:9.210244937
Epoch:1121 - Cost:8.002720675
Epoch:1122 - Cost:9.024832012
Epoch:1123 - Cost:8.645396045
Epoch:1124 - Cost:8.895083555
Epoch:1125 - Cost:8.380420985
Epoch:1126 - Cost:8.455859838
Epoch:1127 - Cost:9.003293781
Epoch:1128 - Cost:8.352959250
Epoch:1129 - Cost:8.995537330
Epoch:1130 - Cost:9.264173793
Epoch:1131 - Cost:8.684981639
Epoch:1132 - Cost:8.678050890
Epoch:1133

...

Epoch:8270 - Cost:nan
Epoch:8271 - Cost:nan
Epoch:8272 - Cost:nan
Epoch:8273 - Cost:nan
Epoch:8274 - Cost:nan
Epoch:8275 - Cost:nan
Epoch:8276 - Cost:nan
Epoch:8277 - Cost:nan
Epoch:8278 - Cost:nan
Epoch:8279 - Cost:nan
Epoch:8280 - Cost:nan
Epoch:8281 - Cost:nan
Epoch:8282 - Cost:nan
Epoch:8283 - Cost:nan
Epoch:8284 - Cost:nan
Epoch:8285 - Cost:nan
Epoch:8286 - Cost:nan
Epoch:8287 - Cost:nan
Epoch:8288 - Cost:nan
Epoch:8289 - Cost:nan
Epoch:8290 - Cost:nan
Epoch:8291 - Cost:nan
Epoch:8292 - Cost:nan
Epoch:8293 - Cost:nan
Epoch:8294 - Cost:nan
Epoch:8295 - Cost:nan
Epoch:8296 - Cost:nan
Epoch:8297 - Cost:nan
Epoch:8298 - Cost:nan
Epoch:8299 - Cost:nan
Epoch:8300 - Cost:nan
Epoch:8301 - Cost:nan
Epoch:8302 - Cost:nan
Epoch:8303 - Cost:nan
Epoch:8304 - Cost:nan
Epoch:8305 - Cost:nan
Epoch:8306 - Cost:nan
Epoch:8307 - Cost:nan
Epoch:8308 - Cost:nan
Epoch:8309 - Cost:nan
Epoch:8310 - Cost:nan
Epoch:8311 - Cost:nan
Epoch:8312 - Cost:nan
Epoch:8313 - Cost:nan
Epoch:8314 - Cost:nan
Epoch:8315

Epoch:8644 - Cost:nan
Epoch:8645 - Cost:nan
Epoch:8646 - Cost:nan
Epoch:8647 - Cost:nan
Epoch:8648 - Cost:nan
Epoch:8649 - Cost:nan
Epoch:8650 - Cost:nan
Epoch:8651 - Cost:nan
Epoch:8652 - Cost:nan
Epoch:8653 - Cost:nan
Epoch:8654 - Cost:nan
Epoch:8655 - Cost:nan
Epoch:8656 - Cost:nan
Epoch:8657 - Cost:nan
Epoch:8658 - Cost:nan
Epoch:8659 - Cost:nan
Epoch:8660 - Cost:nan
Epoch:8661 - Cost:nan
Epoch:8662 - Cost:nan
Epoch:8663 - Cost:nan
Epoch:8664 - Cost:nan
Epoch:8665 - Cost:nan
Epoch:8666 - Cost:nan
Epoch:8667 - Cost:nan
Epoch:8668 - Cost:nan
Epoch:8669 - Cost:nan
Epoch:8670 - Cost:nan
Epoch:8671 - Cost:nan
Epoch:8672 - Cost:nan
Epoch:8673 - Cost:nan
Epoch:8674 - Cost:nan
Epoch:8675 - Cost:nan
Epoch:8676 - Cost:nan
Epoch:8677 - Cost:nan
Epoch:8678 - Cost:nan
Epoch:8679 - Cost:nan
Epoch:8680 - Cost:nan
Epoch:8681 - Cost:nan
Epoch:8682 - Cost:nan
Epoch:8683 - Cost:nan
Epoch:8684 - Cost:nan
Epoch:8685 - Cost:nan
Epoch:8686 - Cost:nan
Epoch:8687 - Cost:nan
Epoch:8688 - Cost:nan
Epoch:8689

Epoch:9018 - Cost:nan
Epoch:9019 - Cost:nan
Epoch:9020 - Cost:nan
Epoch:9021 - Cost:nan
Epoch:9022 - Cost:nan
Epoch:9023 - Cost:nan
Epoch:9024 - Cost:nan
Epoch:9025 - Cost:nan
Epoch:9026 - Cost:nan
Epoch:9027 - Cost:nan
Epoch:9028 - Cost:nan
Epoch:9029 - Cost:nan
Epoch:9030 - Cost:nan
Epoch:9031 - Cost:nan
Epoch:9032 - Cost:nan
Epoch:9033 - Cost:nan
Epoch:9034 - Cost:nan
Epoch:9035 - Cost:nan
Epoch:9036 - Cost:nan
Epoch:9037 - Cost:nan
Epoch:9038 - Cost:nan
Epoch:9039 - Cost:nan
Epoch:9040 - Cost:nan
Epoch:9041 - Cost:nan
Epoch:9042 - Cost:nan
Epoch:9043 - Cost:nan
Epoch:9044 - Cost:nan
Epoch:9045 - Cost:nan
Epoch:9046 - Cost:nan
Epoch:9047 - Cost:nan
Epoch:9048 - Cost:nan
Epoch:9049 - Cost:nan
Epoch:9050 - Cost:nan
Epoch:9051 - Cost:nan
Epoch:9052 - Cost:nan
Epoch:9053 - Cost:nan
Epoch:9054 - Cost:nan
Epoch:9055 - Cost:nan
Epoch:9056 - Cost:nan
Epoch:9057 - Cost:nan
Epoch:9058 - Cost:nan
Epoch:9059 - Cost:nan
Epoch:9060 - Cost:nan
Epoch:9061 - Cost:nan
Epoch:9062 - Cost:nan
Epoch:9063

Epoch:9394 - Cost:nan
Epoch:9395 - Cost:nan
Epoch:9396 - Cost:nan
Epoch:9397 - Cost:nan
Epoch:9398 - Cost:nan
Epoch:9399 - Cost:nan
Epoch:9400 - Cost:nan
Epoch:9401 - Cost:nan
Epoch:9402 - Cost:nan
Epoch:9403 - Cost:nan
Epoch:9404 - Cost:nan
Epoch:9405 - Cost:nan
Epoch:9406 - Cost:nan
Epoch:9407 - Cost:nan
Epoch:9408 - Cost:nan
Epoch:9409 - Cost:nan
Epoch:9410 - Cost:nan
Epoch:9411 - Cost:nan
Epoch:9412 - Cost:nan
Epoch:9413 - Cost:nan
Epoch:9414 - Cost:nan
Epoch:9415 - Cost:nan
Epoch:9416 - Cost:nan
Epoch:9417 - Cost:nan
Epoch:9418 - Cost:nan
Epoch:9419 - Cost:nan
Epoch:9420 - Cost:nan
Epoch:9421 - Cost:nan
Epoch:9422 - Cost:nan
Epoch:9423 - Cost:nan
Epoch:9424 - Cost:nan
Epoch:9425 - Cost:nan
Epoch:9426 - Cost:nan
Epoch:9427 - Cost:nan
Epoch:9428 - Cost:nan
Epoch:9429 - Cost:nan
Epoch:9430 - Cost:nan
Epoch:9431 - Cost:nan
Epoch:9432 - Cost:nan
Epoch:9433 - Cost:nan
Epoch:9434 - Cost:nan
Epoch:9435 - Cost:nan
Epoch:9436 - Cost:nan
Epoch:9437 - Cost:nan
Epoch:9438 - Cost:nan
Epoch:9439

Epoch:9769 - Cost:nan
Epoch:9770 - Cost:nan
Epoch:9771 - Cost:nan
Epoch:9772 - Cost:nan
Epoch:9773 - Cost:nan
Epoch:9774 - Cost:nan
Epoch:9775 - Cost:nan
Epoch:9776 - Cost:nan
Epoch:9777 - Cost:nan
Epoch:9778 - Cost:nan
Epoch:9779 - Cost:nan
Epoch:9780 - Cost:nan
Epoch:9781 - Cost:nan
Epoch:9782 - Cost:nan
Epoch:9783 - Cost:nan
Epoch:9784 - Cost:nan
Epoch:9785 - Cost:nan
Epoch:9786 - Cost:nan
Epoch:9787 - Cost:nan
Epoch:9788 - Cost:nan
Epoch:9789 - Cost:nan
Epoch:9790 - Cost:nan
Epoch:9791 - Cost:nan
Epoch:9792 - Cost:nan
Epoch:9793 - Cost:nan
Epoch:9794 - Cost:nan
Epoch:9795 - Cost:nan
Epoch:9796 - Cost:nan
Epoch:9797 - Cost:nan
Epoch:9798 - Cost:nan
Epoch:9799 - Cost:nan
Epoch:9800 - Cost:nan
Epoch:9801 - Cost:nan
Epoch:9802 - Cost:nan
Epoch:9803 - Cost:nan
Epoch:9804 - Cost:nan
Epoch:9805 - Cost:nan
Epoch:9806 - Cost:nan
Epoch:9807 - Cost:nan
Epoch:9808 - Cost:nan
Epoch:9809 - Cost:nan
Epoch:9810 - Cost:nan
Epoch:9811 - Cost:nan
Epoch:9812 - Cost:nan
Epoch:9813 - Cost:nan
Epoch:9814

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, "trained_vae/model.ckpt")
        
    train_data, test_data = get_weapon_data()
    
    samples = test_data.next_batch(batch_size)
    x_reconstructed = network.encode_and_decode(samples, True)
    print_decoded_tensors_as_dict(test_data, np.concatenate((samples,x_reconstructed), axis=0))


with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, "trained_vae/model.ckpt")

    train_data, test_data = get_weapon_data()
    
    samples = test_data.next_batch(1)
    x_reconstructed_mean = network.encode_and_decode(samples, False)
    print_decoded_tensors_as_dict(test_data, np.concatenate((samples,[x_reconstructed_mean]), axis=0))

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, "trained_vae/model.ckpt")
    
    avg_cost_rand = 0.
    avg_cost = 0.

    _, test_data = get_weapon_data()
    num_samples = test_data.num_examples
    total_batch = int(num_samples / batch_size)

    # Loop over all batches
    for i in range(total_batch):
        batch = test_data.next_batch(batch_size)
        cost = network.calculate_loss(batch)        

        rand_sample = np.random.uniform(low=test_data.standardized_min_values, 
                                       high=test_data.standardized_max_values, 
                                       size=(batch_size,network_architecture["n_input"]))
        cost_rand = network.calculate_loss(rand_sample)

        print("Random Cost = " + "{:.2f}".format(cost_rand))

        #compute average loss/cost
        avg_cost_rand += min(cost_rand,1000) / batch_size
        avg_cost += cost / batch_size
    
    print("Random input average cost = " + "{:.2f}".format(avg_cost_rand))   
    print("Test data input average cost = " + "{:.2f}".format(avg_cost))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_z_distribution(vae_model, title, z_mean=True):
    all_z = np.zeros((batch_size,network_architecture['n_z']))
        
    train_data, test_data = get_weapon_data()
    total_batch = int(train_data.num_examples / batch_size) 

    # Loop over all batches
    for i in range(total_batch):
        batch = train_data.next_batch(batch_size)
        z_dist = vae_model.calculate_z(batch)
        if z_mean:
            z_dist = vae_model.calculate_z_mean(batch)
        all_z = np.vstack((all_z, z_dist))

    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.scatter(all_z[:,0], all_z[:,1])
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    plt.title(title)
    plt.subplot(1,2,2)
    plt.hist2d(all_z[:,0], all_z[:,1], (50, 50), cmap=plt.cm.jet)
    plt.colorbar()
    plt.title(title)
    
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    show_z_distribution(network, "Untrained Latent Space", z_mean=True)
    network = vae.restore(network, "trained_vae/model.ckpt")
    show_z_distribution(network, "Trained Latent Space - Z Mean", z_mean=True)    
    show_z_distribution(network, "Trained Latent Space - Z", z_mean=False)

In [ ]:
# Testing single input in latent space
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, "trained_vae/model.ckpt")
    
    generated = []

    random_val = np.random.normal(size=(1,network_architecture["n_z"]))
    x_test = network.decode_from_latent_space(random_val, False)
    #[generated.append(x) for x in x_test]
    generated.append(x_test)  
    
    train_data, test_data = get_weapon_data()
    print_decoded_tensors_as_dict(train_data, generated)

In [ ]:
# Testing batch input in latent space
with tf.Session(graph=tf.Graph()) as sess:
    network = vae.get_untrained(sess, network_architecture, optimizer, transfer_fct, batch_size)
    network = vae.restore(network, "trained_vae/model.ckpt")
    
    generated = []

    random_val = np.random.normal(size=(batch_size,network_architecture["n_z"]))
    x_test = network.decode_from_latent_space(random_val, True)
    [generated.append(x) for x in x_test]
    
    
    random_val = np.random.normal(size=(batch_size,network_architecture["n_z"]))
    x_test = network.decode_from_latent_space(random_val, True)
    [generated.append(x) for x in x_test]
    
    train_data, test_data = get_weapon_data()
    print_decoded_tensors_as_dict(train_data, generated)